In [1]:
%matplotlib qt
import mne
import numpy as np
import os
from os.path import join
import matplotlib.pyplot as plt

# Loading 

In [23]:
participant_id = '02'
motor_task = 'motoreal'
file_name = f"Data/Raw/{participant_id}/BHIe_{participant_id}_{motor_task}.bdf"

# Load the raw data
raw = mne.io.read_raw_bdf(file_name, preload=True)

Extracting EDF parameters from /home/oem/Documents/HBI-motor_imagery/HBI-motor_imagery/src/preprocessing/Data/Raw/02/BHIe_02_motoreal.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 444415  =      0.000 ...   433.999 secs...


# Processing

1. Events
- motoreal:     Left = 110 / Right = 120
- motorimagery: Left = 210 / Right = 220

In [24]:
from preprocessing.utils import extract_emg_data, extract_ecg_data

In [25]:
# 1. Get the events
events = mne.find_events(raw, initial_event=True) # Store events
events[:, 2] &= (2**9-1) # Modify event data

# 2. Set the EEG reference channels
mne.set_eeg_reference(inst=raw, ref_channels=["EXG1", "EXG2"])

# 3. Get external channels
channel_list = raw.info['ch_names'][:-1] # The full list of channels
ext_channels = channel_list[-8:] # The external channels (EXG1-8)
status = raw.info['ch_names'][-1] # Can be deleted after the events have been extracted

# 4. Extract the datasets (EMG, ECG)
if participant_id != '23':
    emg_data = extract_emg_data(raw_dataset=raw)
else:
    emg_data = extract_emg_data(raw_dataset=raw, inverse=True)


# ECG channels have been inverted in participant 27
if (participant_id != '27'):
    ecg_data = extract_ecg_data(raw_dataset=raw)
else:
    ecg_data = extract_ecg_data(raw_dataset=raw, inverse=True)


# 5. Extract only the EEG data (drop ext. channels)
eeg_data = raw.drop_channels(ext_channels) # not including EXG1-EXG2
eeg_data.drop_channels(status)

103 events found
Event IDs: [ 61441  61550  61560 126976]
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


<RawEDF | BHIe_02_motoreal.bdf, 64 x 444416 (434.0 s), ~217.1 MB, data loaded>

## Preprocess raw data 


In [26]:
from preprocessing.utils import smooth_emg_signal, create_new_raw_dataset

In [27]:
sfreq = raw.info['sfreq']

In [28]:
# EMG DATA (band-pass filter, full-wave rectification, and smooth signal with RMS)
emg_data = mne.filter.filter_data(data=emg_data, sfreq=sfreq, l_freq=10., h_freq=500.)
emg_data = np.abs(emg_data)
emg_data = smooth_emg_signal(emg_data=emg_data, 
                            window_size=int(sfreq/20))


# EEG DATA: Band-pass filter
eeg_data = eeg_data.filter(1., 40., picks=('eeg'), fir_design='firwin')

# ECG DATA: Remove DC offset and Baseline Wander
ecg_data = mne.filter.filter_data(data=ecg_data, sfreq=sfreq, l_freq=0.2, h_freq=40.)

# Combine the data into a new Raw object
raw_dataset = create_new_raw_dataset(eeg_data=eeg_data, ecg_data=ecg_data, emg_data=emg_data)


Setting up band-pass filter from 10 - 5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 10.00
- Lower transition bandwidth: 2.50 Hz (-6 dB cutoff frequency: 8.75 Hz)
- Upper passband edge: 500.00 Hz
- Upper transition bandwidth: 12.00 Hz (-6 dB cutoff frequency: 506.00 Hz)
- Filter length: 1353 samples (1.321 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3381 samples (3.302 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Setting up band-pass filter from 0.2 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.10 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 16897 samples (16.501 sec)



[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Creating RawArray with float64 data, n_channels=67, n_times=444416
    Range : 0 ... 444415 =      0.000 ...   433.999 secs
Ready.


In [17]:


# # Generate and apply BioSemi64 montage
# biosemi_montage = mne.channels.make_standard_montage('biosemi64') # or should we use the standard 10-20?
# raw.set_montage(biosemi_montage)